In [1]:
from models.data import CURVASDataModule, SynthCardDataModule
from models.encoders import DinoViT_16
from models.segmentation import ObjectSpecificSegmentation
from models.uncertainty_measures import make
from models.curvas_metrics import consensus_dice_score
from monai.networks import one_hot

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [2]:
data = SynthCardDataModule(batch_size=1, num_workers=0)

Loading Data: 100%|██████████| 1000/1000 [00:00<00:00, 1619422.39it/s]


In [22]:
def get_normalised_uncertainties(model, batch, trials=500):

    model.slot_attention.probabilistic_sample = True
    model.to('cuda')
    batch = {k: v.to('cuda') for k, v in batch.items()}
    list_entropy = []
    for _ in range(trials):
        with torch.no_grad():
            _, _, _, preds, _, _, _, _ = model.process_batch(batch, 1)

            new_preds = one_hot(preds, num_classes=4)[:, 1:].permute(1, 0, 2, 3)

            list_entropy.append(new_preds)

    all_preds = torch.stack(list_entropy, dim=0).float()

    vars = torch.var(all_preds, dim=0)

    # normalize variance to 0, 1
    vars = (vars - vars.min()) / (vars.max() - vars.min())
    vars *= 100.0 

    return vars

In [5]:
encoder = DinoViT_16()
oss = ObjectSpecificSegmentation.load_from_checkpoint('/vol/bitbucket/bc1623/project/semi_supervised_uncertainty/bash_scripts/runs/lightning_logs/abd_seg/abd_seg/1_ar_t/checkpoints/epoch=429-step=119110.ckpt',
                                encoder=encoder, strict = False)

Using cache found in /homes/bc1623/.cache/torch/hub/facebookresearch_dino_main
/vol/bitbucket/bc1623/project/uncertainty_env/lib/python3.10/site-packages/lightning_fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub 

In [25]:
batch = next(iter(data.test_dataloader()))

ground_truth = batch['labelmap']

batch = {k: v.to('cuda') for k, v in batch.items()}
with torch.no_grad():
    oss.slot_attention.probabilistic_sample = False
    loss, dsc, probs, preds, _, _, _, attn = oss.process_batch(batch, 1)

print(probs.shape)
predictions = preds.cpu().numpy()

torch.Size([25, 4, 1, 512, 512])


In [23]:

print(new_preds.shape)
new_preds = one_hot(preds, num_classes=4)[:, 1:].permute(1, 0, 2, 3)
print(new_preds.shape)

torch.Size([25, 512, 512])
torch.Size([3, 25, 512, 512])


In [26]:
vars = get_normalised_uncertainties(oss, batch, trials=50)
print(vars.shape)

torch.Size([3, 25, 512, 512])


In [30]:
predictions.shape

(25, 1, 512, 512)

In [32]:
# groundtruth: numpy stack list containing the three ground truths [gt1, gt2, gt3]
#                 each gt has the following values: 1: pancreas, 2: kidney, 3: liver
#                 (3, slices, X, Y)
# bin_pred: binarized prediction matrix containing values: {0,1,2,3}
# prob_pred: probability prediction matrix, shape: (3, slices, X, Y), the three being
#             a probability matrix per each class

dice, confidence = consensus_dice_score(ground_truth.cpu().numpy(), predictions.squeeze(1), vars.cpu().numpy())

ValueError: operands could not be broadcast together with shapes (25,512,512) (3,512,512) 

In [3]:
batch = next(iter(data.test_dataloader()))

thresholds = np.array([25.0, 50.0, 75.0, 100.0])
for image_num in range(25):
    ground_truth = batch['ground_truth'][image_num].numpy()

    # segmentation and uncertainty maps from model
    batch = {k: v.to('cuda') for k, v in batch.items()}
    with torch.no_grad():
        loss, dsc, probs, preds, _, _, _, attn = oss.process_batch(batch, 1)
    segmentation = preds[image_num].cpu().numpy()
    uncertainties = get_normalised_uncertainties(oss, batch, trials=500).cpu().numpy()[image_num]
    mask = np.ones_like(ground_truth)
    

KeyError: 'ground_truth'

In [27]:
def calculate_metrics(ground_truth, segmentation, uncertainties, mask, thresholds, object=3):
    # Define Uncertainty Threshold points
    _UNC_POINTs = np.arange(0.0, 100.0 + 1e-6, 100.0 / len(thresholds)).tolist()
    _UNC_POINTs.reverse()
    make(ground_truth==object, segmentation==object, uncertainties, mask, thresholds)

    auc_dice = auc(_UNC_POINTs, make.dice)
    auc_ftp = auc(_UNC_POINTs, make.ftp)
    auc_ftn = auc(_UNC_POINTs, make.ftn)

    metric = (auc_dice + (1 - auc_ftp) + (1 - auc_ftn)) / 3

    return metric

([0.9501359927470535,
  0.9454436450839329,
  0.9404407385348421,
  0.9340788699234844],
 [0.009451796, 0.006301197, 0.0050409576, 0.0],
 [0.004048583, 0.0010293007, 0.0006862005, 0.0])

: 